In [ ]:
%matplotlib notebook

# EL4703 Señales y Sistemas 2020 S2
## Estadísticas del examen parcial

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.special import erf
from sklearn.neighbors import KernelDensity
from sklearn.model_selection import GridSearchCV
from sklearn.mixture import GaussianMixture

Configuración:

In [ ]:
archivo_csv = 'parcial_all.csv' ## Recordar que las columnas tienen que estar separadas por ';'

col_pts = 1   ## Columna en el CSV con los puntos a analizar
total_pts = 46 ## Máximo puntaje obtenible en la columna col_pts


Cargar todos los datos en archivo csv

In [ ]:
data = np.genfromtxt(archivo_csv, delimiter=';')
print("Datos completos en matriz {0} x {1}".format(data.shape[0],data.shape[1]))

# La columna col_pts tiene los puntos totales obtenidos por cada estudiante
pts=data[:,col_pts].reshape(-1,1)
print(pts.shape)

In [ ]:
# Mostrar la densidad probabilistica estimada para un ancho de banda dado manualmente
xplot = np.linspace(0,total_pts,200)[:,np.newaxis]
kde = KernelDensity(kernel="gaussian",bandwidth=3).fit(pts)
log_dens = kde.score_samples(xplot)

plt.figure()
plt.plot(xplot[:,0],np.exp(log_dens))
plt.xlabel("Puntos")
plt.ylabel("p(Puntos)")

In [ ]:
# Estimar el ancho de banda óptimo para estos puntos
params = {'bandwidth': np.logspace(-1,1,200)}
grid = GridSearchCV(KernelDensity(),params,cv=10) # Use 10-fold cross-validation
grid.fit(pts)
kde = grid.best_estimator_
print("Mejor ancho de banda: {0}".format(kde.bandwidth))

In [ ]:
# Muestre la densidad estimada con el mejor ancho de banda
log_dens = kde.score_samples(xplot)

plt.figure()
plt.plot(xplot[:,0],np.exp(log_dens))
plt.xlabel("Puntos")
plt.ylabel("p(Puntos)")

# GMM

Si usamos solo los datos originales, como son relativamente pocos, el GMM queda lejos de la densidad estimada con el kernel gaussiano, por lo que mejor muestreamos el proceso con la densisdad estimada y sacamos muchos más datos, y a partir de esos estimamos el GMM

In [ ]:
gmm = GaussianMixture(n_components=2,covariance_type="spherical",verbose=2)

samples = kde.sample(50000) # Usemos MUCHOS datos
gmm.fit(samples)
print("Weights: ",gmm.weights_)
print("Means: ",gmm.means_)
print("Variances: ", gmm.covariances_)


In [ ]:
pred_dens = (gmm.weights_[0]*norm(gmm.means_[0],np.sqrt(gmm.covariances_[0])).pdf(xplot[:,0])
             + gmm.weights_[1]*norm(gmm.means_[1],np.sqrt(gmm.covariances_[0])).pdf(xplot[:,0]))

plt.figure()
plt.plot(xplot[:,0],np.exp(log_dens),'-b',label="KDE")
plt.plot(xplot[:,0],pred_dens,'-k',label="GMM",linewidth=3)
plt.plot(xplot[:,0],gmm.weights_[0]*norm.pdf(xplot[:,0],gmm.means_[0],np.sqrt(gmm.covariances_[0])),'-r')
plt.plot(xplot[:,0],gmm.weights_[1]*norm.pdf(xplot[:,0],gmm.means_[1],np.sqrt(gmm.covariances_[1])),'-g')

plt.xlabel("Puntos")
plt.ylabel("p(Puntos)")
plt.legend(loc='upper left')


En el experimento anterior definimos a priori que queríamos dos gaussianas en el GMM, pero ¿será mejor otro modelo?.  Lo siguiente busca exhaustivamente el número óptimo de gaussianas a utilizar

In [ ]:
lowest_bic = np.infty
best_n_components = -1
bic = []

n_components_range = range(1, 7)
cv_type = 'spherical'
for n_components in n_components_range:
    # Fit a Gaussian mixture with EM
    gm = GaussianMixture(n_components=n_components,covariance_type=cv_type)
    gm.fit(samples)
    bic.append(gm.bic(samples)) # Bayesian Information Criterion
    #bic.append(gm.aic(samples)) # Akaike Information Criterion
    print("  bic con {0} componentes = {1}".format(n_components,bic[-1]))
    if bic[-1] < lowest_bic:
        lowest_bic = bic[-1]
        best_gmm = gm
        best_n_components = n_components
        
print("Mejor número de componentes en GMM: ",best_n_components)


In [ ]:
def Phi(z):
    return 0.5*(1 + erf(z/np.sqrt(2)))

t=np.linspace(0,total_pts,1000)[:,np.newaxis]

f=np.argmin(gmm.means_)
s=np.argmax(gmm.means_)

k=np.sqrt(np.pi)/2
f0=gmm.weights_[f]*Phi((-t+gmm.means_[f])/np.sqrt(gmm.covariances_[f]))
f1=gmm.weights_[s]*Phi((t-gmm.means_[s])/np.sqrt(gmm.covariances_[s]))

plt.figure()
plt.plot(t,f0,label="left gaussian error")
plt.plot(t,f1,label="right gaussian error")
plt.plot(t,f0+f1,label="classification error")
plt.xlabel("Decision Threshold")
plt.ylabel("Error probability")
plt.legend(loc='upper center')

idx = np.argwhere(np.diff(np.sign(f1 - f0).flatten('F')))
threshold = t.item(int(idx))
clferror = f0.item(int(idx))+f1.item(int(idx))
print("Minimal classification error at {0} pts -> {1}% = {2}".format(threshold,100*threshold/total_pts,clferror))


In [ ]:
np.mean(pts)